In [1]:
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")
from appaloosa import RunLC
import os
import numpy as np
import pyke as pk
import pandas as pd
#oid=211397837
oid=211389537
oid=211395992
oid = 211423010
loc = 'stars_shortlist/M67'
tpf_loc = 'stars_shortlist/M67/ktwo{}-c05_lpd-targ.fits.gz'.format(oid)
lc_loc = 'stars_shortlist/M67/hlsp_everest_k2_llc_{}-c05_kepler_v2.0_lc.fits'.format(oid)



flux = pd.read_csv('stars_shortlist/M67/results/run_05/{}_flux.csv'.format(oid))#,usecols=['time','flux_gap','flux_model'])
flares = pd.read_csv('stars_shortlist/M67/results/run_05/{}_flares.csv'.format(oid))
#flux.plot(x='time',y=['flux_gap','flux_model'])
#flux.plot(x='time',y='flux_model')
# for index, row in flares.iterrows():
#     print(flux.loc[row.istart:row.istop])
flares

,Unnamed: 0,ampl,dur,ed,ederr,istart,istop,istart_no_sys_no_thruster,istop_no_sys_no_thruster,corrected,myed,myenerg,myenerg_err,L_bol,L_bol_err,falsepos_corrected,ED_true,count_true,energ_true
0,0,0.051297,0.040863,68.954999,15.252132,2504,2506,2504,2506,2.2,70.946182,3.867016e+33,1.936872e+33,1.493933e+33,2.235941e+32,0,130.122725,2.2,7.092511e+33


In [3]:
import k2flix
tpf2 = k2flix.TargetPixelFile('stars_shortlist/M67/ktwo{}-c05_lpd-targ.fits.gz'.format(oid))
for i in range(flares.istart_no_sys_no_thruster.iloc[0],flares.istop_no_sys_no_thruster.iloc[0]):
    print(tpf2.quality_flags(i))
    

[]
['Thruster firing']


In [7]:
#import sys
#!{sys.executable} -m pip install lightkurve


#!echo $PATH
#!type pip
#%cd /Enter/your/prefered/path/here/
#sys.path
!pip3 list

DEPRECATION: The default format will switch to columns in the future. You can use --format=(legacy|columns) (or define a format=(legacy|columns) in your pip.conf under the [list] section) to disable this warning.
alabaster (0.7.10)
apparmor (2.10.3)
appdirs (1.4.3)
asciitree (0.3.3)
asn1crypto (0.22.0)
astropy (1.3.2)
astroquery (0.3.6.post2)
attrs (17.3.0)
Automat (0.6.0)
Babel (2.5.1)
backports-abc (0.5)
bcrypt (3.1.4)
beautifulsoup4 (4.6.0)
bkcharts (0.2)
bleach (2.1.2)
bmcmc (0.1.0)
bokeh (0.12.6)
bottle (0.12.13)
certifi (2017.4.17)
cffi (1.10.0)
chardet (3.0.4)
cheroot (5.4.0)
CherryPy (10.2.1)
configobj (5.0.6)
constantly (15.1.0)
corner (2.0.1)
coverage (4.4.2)
cryptography (2.0.2)
cssselect (1.0.1)
cupshelpers (1.0)
cycler (0.10.0)
Cython (0.25.2)
decorator (4.1.2)
docrepr (0.1.0)
docutils (0.14)
ebfpy (0.0.14)
entrypoints (0.2.3)
flup (1.0.3.dev20161029)
gatspy (0.3)
Genshi (0.7)
gevent (1.2.2)
greenlet (0.4.12)
h5py (2.7.1)
HeapDict (1.0.0)
html5lib (0.999999999)
hyperlink (

In [4]:
tpf2.save_movie(output_fn='pxl_{}.gif'.format(oid), start=flares.istart_no_sys_no_thruster.iloc[0], 
                stop=flares.istop_no_sys_no_thruster[0], step=1,
               binning=1, fps=3, dpi=None, stretch='linear',
               min_cut=None, max_cut=None, min_percent=60, max_percent=100,
               cmap='hot', time_format='frameno', show_flags=True, data_col='FLUX',
               ignore_bad_frames=True,)

 33%|███▎      | 1/3 [00:00<00:00,  9.89it/s]

Creating pxl_211423010.gif


100%|██████████| 3/3 [00:00<00:00,  9.07it/s]


In [5]:

tpf = pk.KeplerTargetPixelFile('stars_shortlist/M67/ktwo{}-c05_lpd-targ.fits.gz'.format(oid))
apmask = tpf.pipeline_mask*0.01
#d = tpf.flux*tpf.pipeline_mask
tpf.time


import math
import timeit
def find_nearest(array,value):
    idx = np.searchsorted(array, value, side="left")
    if idx > 0 and (idx == len(array) or math.fabs(value - array[idx-1]) < math.fabs(value - array[idx])):
        return array[idx-1]
    else:
        return array[idx]
    
flux['tpf_time'] = 'Nan'
for id_,row in flux.iterrows():
    try:
        flux['tpf_time'].iloc[id_] = find_nearest(tpf.time, row.time)
        #timeit.timeit(find_nearest(tpf.time, row.time))
        #flux['tpf_time'].iloc[id_] = tpf.time[np.argmin(abs(tpf.time[id_:]-row.time))]
        #print(tpf.time[max(0,id_-5):min(id_+5,len(tpf.time))])
    except ValueError:
        pass
    

flux.iloc[645:655]

flux['isflare'] = False
print(flux.head())
for id_, row in flares.iterrows():
    flux['isflare'].iloc[row.istart:row.istop+1] = True
flux.to_csv('{}_tpf_times_FLUX.csv'.format(oid))
#flux.iloc[1574:1603]

flux_648 = pd.read_csv('211410963_tpf_times_FLUX.csv')
flux_648.time = flux_648.time.round(decimals=4)
flux.time = flux.time.round(decimals=4)
a = flux.merge(flux_648, on='time')
#print(flux.head())
#print(flux_648.head())
#print(a.head())
s = a[(a.isflare_x == a.isflare_y)]
#print(s.info())
p = a[(a.isflare_x == True)]
#print(p.info())
t = s[s.isflare_x == True]
#print(t)

select_isflare = flux[flux.isflare==True]
isflare_tpf_times = select_isflare.tpf_time.tolist()

for i in isflare_tpf_times[35:65]:
    id_ = np.where(tpf.time == i)[0][0]
    print(id_)

AttributeError: 'module' object has no attribute 'KeplerTargetPixelFile'

In [1]:
import lightkurve


ImportError: Module 'dbm' is not installed.
Use:
  sudo zypper install python3-dbm
to install it.